##MT5 Using Simple Transformers

In [2]:
!pip install folium==0.2.1

     |████████████████████████████████| 69 kB 3.6 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=c4e795da4940b13d01ea679067d85365b8c81ce754e79d7439f737ab011ecf35
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [3]:
! pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 49.1 MB/s 
     |████████████████████████████████| 134 kB 52.0 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 127 kB 55.4 MB/s 
     |████████████████████████████████| 144 kB 52.4 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 42.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
! pip install transformers

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
     |████████████████████████████████| 596 kB 41.3 MB/s 
     |████████████████████████████████| 6.6 MB 36.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 4.8 MB/s 
     |████████████████████████████████| 10.1 MB 21.2 MB/s 
     |████████████████████████████████| 1.7 MB 36.4 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 40.7 MB/s 
     |████████████████████████████████| 144 kB 38.1 MB/s 
     |████████████████████████████████| 181 kB 32.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 164 kB 37.6 MB/s 
     |████████████████████████████████| 111 kB 39.6 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 4.3 MB 22.6 MB/s 
     |████████████████████████████████| 130 kB 39.0 MB/s 
     |████████████████████████████████| 793 kB 39.6 MB/s 
     |████████████████████████████████| 428 kB 37.1 MB/s 
     |████████████████████████████████| 1.1 MB 6.8 MB/s 
     |████████████████████████████████| 130 kB 34.4 MB/s 
     |███████████████

In [1]:
import pandas as pd
import json
import csv
from sklearn.model_selection import train_test_split
import logging
from simpletransformers.t5 import T5Model, T5Args
from datasets import load_dataset, load_metric

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###1) Loading the data, Preparing and Saving it for easier access later.

This Section is needed only once.

In [6]:
datasets =load_dataset('json',
                       data_files={'train': '/content/drive/MyDrive/GP-2022/qrcd_v1.1_train.jsonl',
                                   'validation': '/content/drive/MyDrive/GP-2022/qrcd_v1.1_dev.jsonl'})

Using custom data configuration default-703fabdad370c8d8
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-703fabdad370c8d8/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
        num_rows: 710
    })
    validation: Dataset({
        features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
        num_rows: 109
    })
})

In [9]:
type(datasets["train"])

datasets.arrow_dataset.Dataset

In [10]:
datasets["train"]

Dataset({
    features: ['pq_id', 'passage', 'surah', 'verses', 'question', 'answers'],
    num_rows: 710
})

In [11]:
df_train = pd.DataFrame(datasets["train"])
df_val = pd.DataFrame(datasets["validation"])
df_train.head()

,pq_id,passage,surah,verses,question,answers
0,2:8-16_364,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,2,8-16,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'أولئك الذين اشتروا الضلالة بالهدى',..."
1,2:174-176_364,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,2,174-176,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,[{'text': 'أولئك الذين اشتروا الضلالة بالهدى و...
2,14:47-52_364,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,14,47-52,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'ليجزي الله كل نفس ما كسبت', 'start_..."
3,17:12-17_364,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,17,12-17,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'كل إنسان ألزمناه طائره في عنقه', 's..."
4,18:29-31_364,وقل الحق من ربكم فمن شاء فليؤمن ومن شاء فليكفر...,18,29-31,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,"[{'text': 'من شاء فليؤمن ومن شاء فليكفر', 'sta..."


In [12]:
df_val.head()

,pq_id,passage,surah,verses,question,answers
0,9:60-61_316,إنما الصدقات للفقراء والمساكين والعاملين عليها...,9,60-61,ما هي مصارف الزكاة؟,[{'text': 'للفقراء والمساكين والعاملين عليها و...
1,2:178-179_400,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,2,178-179,متى يحل الإسلام دم الشخص؟,[{'text': 'كتب عليكم القصاص في القتلى الحر بال...
2,2:190-194_400,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,2,190-194,متى يحل الإسلام دم الشخص؟,[{'text': 'قاتلوا في سبيل الله الذين يقاتلونكم...
3,2:243-245_400,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,2,243-245,متى يحل الإسلام دم الشخص؟,"[{'text': 'قاتلوا في سبيل الله', 'start_char':..."
4,4:74-76_400,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,4,74-76,متى يحل الإسلام دم الشخص؟,"[{'text': 'الذين آمنوا يقاتلون في سبيل الله', ..."


In [13]:
df_train.question[45]

'ما هي انواع الحيوانات التي ذكرت في القرآن؟'

In [14]:
df_train.answers[45] #list of dictionaries

[{'start_char': 98, 'text': 'العجل'}, {'start_char': 348, 'text': 'السلوى'}]

In [15]:
def prepare_dataset(dataset):
  """ returns a prepared copy of the dataset given after preparing it and getting the columns we want for the mt5 model"""
  dataset_pepared = dataset.copy()
  dataset_pepared = dataset_pepared.explode('answers').reset_index(drop=True) # make a row for each answer dict
  dataset_pepared = dataset_pepared.join(pd.json_normalize(dataset_pepared.answers)).drop(columns=['answers']) #unzip the answer dict into 2 columns and drop the original
  dataset_pepared = dataset_pepared[["passage", "question", "text"]] # for mt5 model we only need  these columns
  return dataset_pepared


In [16]:
# use explode to give each dict in a list a separate row
#train = df_train.copy()
#train = train.explode('answers').reset_index(drop=True)
#train = train.join(pd.json_normalize(train.answers)).drop(columns=['answers'])
train = prepare_dataset(df_train)
train.head()

,passage,question,text
0,ومن الناس من يقول آمنا بالله وباليوم الآخر وما...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى
1,إن الذين يكتمون ما أنزل الله من الكتاب ويشترون...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...
2,فلا تحسبن الله مخلف وعده رسله إن الله عزيز ذو ...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,ليجزي الله كل نفس ما كسبت
3,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,كل إنسان ألزمناه طائره في عنقه
4,وجعلنا الليل والنهار آيتين فمحونا آية الليل وج...,لماذا سيُحاسب ويُعذب الضال يوم القيامة ان كان ...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عليها


In [17]:
val = prepare_dataset(df_val)
val.head()

,passage,question,text
0,إنما الصدقات للفقراء والمساكين والعاملين عليها...,ما هي مصارف الزكاة؟,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...
1,يا أيها الذين آمنوا كتب عليكم القصاص في القتلى...,متى يحل الإسلام دم الشخص؟,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...
2,وقاتلوا في سبيل الله الذين يقاتلونكم ولا تعتدو...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله الذين يقاتلونكم
3,ألم تر إلى الذين خرجوا من ديارهم وهم ألوف حذر ...,متى يحل الإسلام دم الشخص؟,قاتلوا في سبيل الله
4,فليقاتل في سبيل الله الذين يشرون الحياة الدنيا...,متى يحل الإسلام دم الشخص؟,الذين آمنوا يقاتلون في سبيل الله


Now we save the dataframes to csv files for easier loading and accessing later.

In [18]:
train.to_csv('/content/drive/MyDrive/GP-2022/training-data-bp.csv')
val.to_csv('/content/drive/MyDrive/GP-2022/validation-data-bp.csv')

### Reading the Data, preprocessing it.

In [3]:
train = pd.read_csv("/content/drive/MyDrive/GP-2022/training-data-bp.csv")
val = pd.read_csv("/content/drive/MyDrive/GP-2022/validation-data-bp.csv")

Any preprocessing should happen here.

### Prepare the Data for the mt5 model:
- Simple transformers implementation of mT5 model expects a data to be a dataframe with 3 columns: \<prefix>, \<input_text>, \<target_text>
- \<prefix>: A string indicating the task to perform. (E.g. “question”, “stsb”, “summarization”)

- \<input_text>: The input text sequence (we will use question and context as input_text )

- \<target_text>: The target sequence (we will use text as output_text )

In [4]:
def prepare_dataset_mt5(dataset):
  """ returns a dataframe of the data in the required shape for the mt5 model which is 3 columns <prefix>, <input_text>, <target_text>"""
  dataset_mt5 = pd.DataFrame()
  dataset_mt5['input_text'] = dataset.apply(lambda row: "question: %s  context: %s " % (row['question'], row['passage']), axis=1)
  dataset_mt5['target_text'] = dataset.apply(lambda row: "%s " % (row['text']), axis=1)
  dataset_mt5['prefix'] = "answer question"
  return dataset_mt5

In [5]:
#train_mt5 = pd.DataFrame()
#train_mt5['input_text'] = train.apply(lambda row: "question: %s  context: %s </s>" % (row['question'], row['passage']), axis=1)
#train_mt5['target_text'] = train.apply(lambda row: "%s </s>" % (row['text']), axis=1)
#train_mt5['prefix'] = "answer question"
train_mt5 = prepare_dataset_mt5(train)
train_mt5.head()

,input_text,target_text,prefix
0,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,أولئك الذين اشتروا الضلالة بالهدى,answer question
1,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...,answer question
2,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,ليجزي الله كل نفس ما كسبت,answer question
3,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,كل إنسان ألزمناه طائره في عنقه,answer question
4,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عل...,answer question


In [6]:
val_mt5 = prepare_dataset_mt5(val)
val_mt5.head()

,input_text,target_text,prefix
0,question: ما هي مصارف الزكاة؟ context: إنما ا...,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...,answer question
1,question: متى يحل الإسلام دم الشخص؟ context: ...,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...,answer question
2,question: متى يحل الإسلام دم الشخص؟ context: ...,قاتلوا في سبيل الله الذين يقاتلونكم,answer question
3,question: متى يحل الإسلام دم الشخص؟ context: ...,قاتلوا في سبيل الله,answer question
4,question: متى يحل الإسلام دم الشخص؟ context: ...,الذين آمنوا يقاتلون في سبيل الله,answer question


### Train the Model

In [7]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [8]:
train_mt5.astype(str)


,input_text,target_text,prefix
0,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,أولئك الذين اشتروا الضلالة بالهدى,answer question
1,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,أولئك الذين اشتروا الضلالة بالهدى والعذاب بالم...,answer question
2,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,ليجزي الله كل نفس ما كسبت,answer question
3,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,كل إنسان ألزمناه طائره في عنقه,answer question
4,question: لماذا سيُحاسب ويُعذب الضال يوم القيا...,من اهتدى فإنما يهتدي لنفسه ومن ضل فإنما يضل عل...,answer question
...,...,...,...
856,question: هل تحدث سيدنا محمد مع الجن؟ context...,قل أوحي إلي أنه استمع نفر من الجن,answer question
857,question: ما هي كفارة الظهار؟ context: قد سمع...,تحرير رقبة من قبل أن يتماسا ذلكم توعظون به وال...,answer question
858,question: من هم المطففون؟ context: ويل للمطفف...,الذين إذا اكتالوا على الناس يستوفون. وإذا كالو...,answer question
859,question: ما هو فضل ليلة القدر؟ context: إنا ...,ليلة القدر خير من ألف شهر. تنزل الملائكة والرو...,answer question


In [9]:
val_mt5.astype(str)

,input_text,target_text,prefix
0,question: ما هي مصارف الزكاة؟ context: إنما ا...,للفقراء والمساكين والعاملين عليها والمؤلفة قلو...,answer question
1,question: متى يحل الإسلام دم الشخص؟ context: ...,كتب عليكم القصاص في القتلى الحر بالحر والعبد ب...,answer question
2,question: متى يحل الإسلام دم الشخص؟ context: ...,قاتلوا في سبيل الله الذين يقاتلونكم,answer question
3,question: متى يحل الإسلام دم الشخص؟ context: ...,قاتلوا في سبيل الله,answer question
4,question: متى يحل الإسلام دم الشخص؟ context: ...,الذين آمنوا يقاتلون في سبيل الله,answer question
...,...,...,...
123,question: هل سمح الإسلام بحرية الاعتقاد بالدخو...,من اهتدى فلنفسه ومن ضل فإنما يضل عليها,answer question
124,question: هل سمح الإسلام بحرية الاعتقاد بالدخو...,قل يا أيها الكافرون. لا أعبد ما تعبدون. ولا أن...,answer question
125,question: هل يؤثم الحاكم الذي لا يحكم بما أنزل...,من لم يحكم بما أنزل الله فأولئك هم الكافرون,answer question
126,question: هل يؤثم الحاكم الذي لا يحكم بما أنزل...,من لم يحكم بما أنزل الله فأولئك هم الظالمون,answer question


In [23]:
model_args = T5Args()
model_args.max_seq_length = 196
model_args.train_batch_size = 3
model_args.eval_batch_size = 3
model_args.num_train_epochs = 1
model_args.evaluate_during_training = False
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.preprocess_inputs = True
model_args.overwrite_output_dir = True
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 mixed tasks"

model = T5Model("mt5", "google/mt5-base", args=model_args)

In [11]:
#import torch
#torch.cuda.empty_cache()
#import tensorflow as tf
#tf.test.gpu_device_name()
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

!nvidia-smi

Thu Mar 31 13:05:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Tesla K80
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:386: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [24]:
# Train the model
model.train_model(train_mt5)

# Optional: Evaluate the model. We'll test it properly anyway.
#results = model.eval_model(val_mt5, verbose=True)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/861 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_model: Training started


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

global_step,▁▃▅▆█
lr,▁▁▁▁▁
Training loss,nan
global_step,250
lr,0.001


Running Epoch 0 of 1:   0%|          | 0/287 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model: Training of google/mt5-base model complete. Saved to outputs/.


(287, 3.95247122634994)

In [22]:
results

{'eval_loss': nan}

In [30]:
to_predict = [
    prefix + ": " + str(input_text)
    for prefix, input_text in zip(val_mt5["prefix"].tolist(), val_mt5["input_text"].tolist())
]
to_predict[45]
#type(to_predict[0])

'answer question: question: ماذا يشمل الإحسان؟  context: ولئن سألتهم من خلق السماوات والأرض وسخر الشمس والقمر ليقولن الله فأنى يؤفكون. الله يبسط الرزق لمن يشاء من عباده ويقدر له إن الله بكل شيء عليم. ولئن سألتهم من نزل من السماء ماء فأحيا به الأرض من بعد موتها ليقولن الله قل الحمد لله بل أكثرهم لا يعقلون. وما هذه الحياة الدنيا إلا لهو ولعب وإن الدار الآخرة لهي الحيوان لو كانوا يعلمون. فإذا ركبوا في الفلك دعوا الله مخلصين له الدين فلما نجاهم إلى البر إذا هم يشركون. ليكفروا بما آتيناهم وليتمتعوا فسوف يعلمون. أولم يروا أنا جعلنا حرما آمنا ويتخطف الناس من حولهم أفبالباطل يؤمنون وبنعمة الله يكفرون. ومن أظلم ممن افترى على الله كذبا أو كذب بالحق لما جاءه أليس في جهنم مثوى للكافرين. والذين جاهدوا فينا لنهدينهم سبلنا وإن الله لمع المحسنين. '

In [31]:
model.predict(to_predict)

Generating outputs:   0%|          | 0/43 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3524: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f58b1887200> on tensor will not be serial

Decoding outputs:   0%|          | 0/128 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f58fbf98050> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f58fbf98200> on tensor will not be serialized.  If this is expected, you can decorate the function with @torch.utils.hooks.unserializable_hook to suppress this warning
  "to suppress this warning".format(repr(hook)))
/usr/local/lib/python3.7/dist-packages/torch/utils/hooks.py:62: UserWarning: backward hook <function TorchHistory._hook_variable_gradient_stats.<locals>.<lambda> at 0x7f58fbf983b0> on tensor will not be serialized.  If this is exp

['منكم والذين يؤمن بالله ويمن للمؤمنين',
 'إنا كتب علي وحيه من أخيه شيء فاتباع بالمعروف',
 'لله لا يحل الإسلام مع الشهر الحرام',
 'من الذيييدرض الله قرضا',
 'نعمتيهذه',
 'من يدخلنا من سبيل الله',
 'يطع الرسول',
 'إنما جاءهم سبت من الأمين',
 'آتى يهاجرون منهم كبكم',
 'يهاجروا في سبيل الله',
 'من فوا من عند ذلك لهم خزي في الدنيا',
 'أمطرا أن يصلبوا',
 'إن يعودوا',
 'إن تبتم أنت تبتم فهو خير لكم وإن تسويتم',
 'قالوا منكم',
 'الذين آمتوا',
 'حرم الله زين لهم سوء أعمالهم',
 'ليكفروا',
 'متى يحل الله إلا بالحق ومن يزك',
 'لله',
 'قرآن',
 'ožičaين ثم يكم تهمكم بهم ثم مثلى',
 'موسم',
 'لا توكين',
 'ستور علي',
 'تاب من هذه الشجرة فتكونا من الظالمين',
 'الجود',
 '',
 'لا تسفكون دماءكم ولا تسفكون',
 'كتب عليكم القصاص',
 'من يدخل ذلك',
 'الذين طلقتم به من خطبة أشهر',
 'إن ي يصابهم في سبيل الله',
 'إنماذا يشملهم',
 'يؤمن بالله ولا يؤمن الله ولا يظلم مثقال',
 'إنثى وهو محسن',
 'قال الله إني عليكم بينيtil  فيه ن إليjing',
 'الرسول',
 'فر العالمي',
 'إنما ما على المحسنين من سبيل',
 "''من حولكم من إسرا